In [2]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import seaborn as sns

import pydot



In [4]:
data_train = pd.read_csv('/content/trac2_iben_train.csv')
data_train.head()

ID  ... Sub-task B
0    C45.688  ...       NGEN
1  C45.635.5  ...       NGEN
2    C47.103  ...        GEN
3    C68.147  ...       NGEN
4    C45.615  ...       NGEN

[5 rows x 4 columns]

In [5]:

data_test = pd.read_csv('/content/trac2_iben_dev.csv')


In [6]:
data_train.drop(['ID','Sub-task B'], axis=1, inplace=True)
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)




In [7]:
NAG_train = data_train[(data_train['Sub-task A']=='NAG')]
OAG_train = data_train[(data_train['Sub-task A']=='CAG')]
CAG_train = data_train[(data_train['Sub-task A']=='OAG')]

In [8]:
NAG_test = data_test[(data_test['Sub-task A']=='NAG')]
OAG_test= data_test[(data_test['Sub-task A']=='CAG')]
CAG_test = data_test[(data_test['Sub-task A']=='OAG')]

In [9]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_train["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_train["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_train["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

#NAG_train.head()

In [10]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_test["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_test["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_test["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_test.head()

Text  Sub-task A
0   Ek dom sothik kotha bolecho jhekane theka uthe...           0
2                                    DARUN AKDOM THIK           0
5                                     Kadam thik dada           0
12                               Supar bolla cho dada           0
15                                              Pagle           0

In [11]:
data_train = pd.concat([NAG_train,
                 OAG_train,
                  CAG_train
                ],axis=0)

data_train.reset_index(inplace=True)

In [12]:
data_test = pd.concat([NAG_test,
                  OAG_test,
                  CAG_test
                 ],axis=0)

data_test.reset_index(inplace=True)

In [13]:

data_test = data_test.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [14]:

data_train = data_train.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [15]:
data_train

index                                               Text  label
0         0                                      Dada taratari      0
1         1                              Tumi korbe Amar sathe      0
2         4  পতিতাদের চরিত্র রাজনৈতিক নেতাদের থেকে হাজার গু...      0
3         5                                  Thik bolecho bhai      0
4         7                                             অমানুষ      0
...     ...                                                ...    ...
3821   3795                             Ora gushti sudho pagol      2
3822   3798  এরা সব এক একটা বেটিচোদ।\nএদের জন্য সমাজ নষ্ট হ...      2
3823   3808                                     মাদার চোদ রানু      2
3824   3814  @হিয়াশা জামান হীর orai amr maal khaa. Tate To...      2
3825   3820                                          বাল মন্ডল      2

[3826 rows x 3 columns]

In [16]:
data_train.drop(['index'], axis=1, inplace=True)

In [17]:
X = data_train.Text
Y = data_train.label
le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

In [18]:
X_train,X_valid,y_train,y_valid = train_test_split(X,Y,test_size=0.15)

In [19]:

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(len(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(len(y_valid)))

Train data len:3252
Class distribution3252
Valid data len:574
Class distribution574


In [20]:
num_words = 50000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(X_train)


print(str(tokenizer.texts_to_sequences(['xyz how are you'])))

[[1, 1, 183, 160]]


In [21]:

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )



x_train = pad_sequences(x_train, padding='post', maxlen=150)
x_valid = pad_sequences(x_valid, padding='post', maxlen=150)
x_test = pad_sequences(x_test, padding='post', maxlen=150)



train_labels = np.asarray(y_train )
valid_labels = np.asarray( y_valid)

test_labels = np.asarray(data_test['label'].tolist())

#print('Train data len:'+str(len(x_train)))
#print('Class distribution'+str(Counter(train_labels)))

#print('Validation data len:'+str(len(x_valid)))
#print('Class distribution'+str(Counter(valid_labels)))

#print('Test data len:'+str(len(x_test)))
#print('Class distribution'+str(Counter(test_labels)))


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

In [22]:

max_features =50000
embedding_dim =16
sequence_length = 150


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 64, input_length=sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 30
history_bi = model.fit(x_train,train_labels, epochs=num_epochs, validation_data=(x_valid, valid_labels))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           3200000   
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 128)          66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 3,311,489
Trainable params: 3,311,489
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
102/102 [==============================] - 39s 319ms/step - loss: 0.0000e+00 - accuracy: 0.5322 - val

In [23]:

x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=150)


In [24]:
print("Generate predictions for all samples")
predictions = model.predict(x_test)

Generate predictions for all samples


In [25]:

data_test['pred_sentiment']= predictions

In [26]:

labels = [0,1,2]
    
print(classification_report(data_test['label'].tolist(),data_test['pred_sentiment'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.71      0.86      0.78       522
           1       0.30      0.44      0.36       218
           2       0.00      0.00      0.00       217

    accuracy                           0.57       957
   macro avg       0.34      0.43      0.38       957
weighted avg       0.45      0.57      0.50       957

